# Report

For the next stage, we'd like you to try to tackle a ML task. Please take a look at this kaggle-like challenge and see what you can do with it.

https://kelvins.esa.int/proba-v-super-resolution/home/

In particular, what we're looking for is
- Your analysis and understanding of the provided problem, challenge, data and results
- Quality and creativity of the solution
- Thought process and ability to convey it to us in words, tables, plots and/or visualizations
- Readability and usability of the code
-Feel free to bring your personal touch for problem-solving. Thinking outside of the box is a great way to impress us and making sure you're going to the next phase of the recruitment process! (;

Submitting solutions are appreciated and recommended.
Ranking high is potentially impressive but not necessarily expected given the time and other constraints.

There is no hard deadline for the task. We expect you to work occasionally on it in case you are busy with other affairs (which we suppose you are).

You are free to use your preferred choice of language or frameworks or external data sources if you so see fit.

Expected deliverables are:
- code, preferably in github form
- Report describing your problem analysis, approach, results. Can be in pdf, readme markdown, ipynb form, a mix thereof, or whatever you feel is good at conveying information.

Please do not hesitate to contact us if you have questions or concerns.